In [ ]:
import random
import time
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
IPRange = 50
IPSize = 15
timeout = 500
routeQtd = 8
destinationHost = "192.168.1.100"
sourceHost = "192.168.1.1"
selectionQtd = 4
lastDestinationOc = 100
lastSourceOc = 1

In [ ]:
class Router:
    def __init__(self, ip, lastBinOc, nextTime, fitness):
        self.ip = ip
        self.nextRtd = None
        self.lastBinOc = lastBinOc
        self.nextTime = nextTime
        self.fitness = fitness
        self.jitter = 0

In [ ]:
class Route:
    def __init__(self):
        self.firstRouter = None
        self.routeTime = 0
        self.fitnessSum = 0.0

In [ ]:
def remove_repeated(way):
    for i in range(routeQtd):
        iterator = way[i].firstRouter
        while iterator.nextRtd is not None:
            comparator = iterator.nextRtd
            if comparator.ip == iterator.ip:
                iterator.nextRtd = comparator.nextRtd
                del comparator
            else:
                iterator = iterator.nextRtd

In [ ]:
def update_total_latencies(way):
    for i in range(routeQtd):
        iterator = way[i].firstRouter
        way[i].routeTime = 0
        last_time = None
        while iterator.nextRtd is not None:
            if last_time is not None:
                iterator.jitter = abs(iterator.nextTime - last_time)
            last_time = iterator.nextTime
            way[i].routeTime += iterator.nextTime
            iterator = iterator.nextRtd

In [ ]:
def evaluate_fitness(way):
    for i in range(routeQtd):
        iterator = way[i].firstRouter
        way[i].fitnessSum = 0

        while iterator.nextRtd is not None:
            latency_fitness = 1.0 / iterator.nextTime
            jitter_fitness = 1.0 / (iterator.jitter + 0.00001)
            iterator.fitness = latency_fitness + jitter_fitness
            iterator = iterator.nextRtd

        way[i].fitnessSum = 1.0 / (way[i].routeTime + 0.00001)

In [ ]:
def dijkstra_shortest_path(graph, source, destination):
    shortest_path = nx.dijkstra_path(graph, source=source, target=destination, weight='weight')
    return shortest_path

In [ ]:
def build_graph(way):
    G = nx.Graph()

    for i, route in enumerate(way):
        aux = route.firstRouter

        while aux is not None:
            G.add_node(aux.ip)

            if aux.nextRtd is not None:
                G.add_edge(aux.ip, aux.nextRtd.ip, weight=aux.nextTime)

            aux = aux.nextRtd

In [ ]:
def build_graph(way):
    G = nx.Graph()

    for i, route in enumerate(way):
        aux = route.firstRouter

        while aux is not None:
            G.add_node(aux.ip)

            if aux.nextRtd is not None:
                G.add_edge(aux.ip, aux.nextRtd.ip, weight=aux.nextTime)

            aux = aux.nextRtd

    # Add source and destination nodes
    G.add_node("192.168.1.1")
    G.add_node("192.168.1.100")

    # Connect source and destination nodes to random nodes in the graph
    random_node_1 = random.choice(list(G.nodes()))
    random_node_2 = random.choice(list(G.nodes()))

    G.add_edge("192.168.1.1", random_node_1, weight=0)
    G.add_edge(random_node_2, "192.168.1.100", weight=0)

    return G

In [ ]:
def plot_routes_with_color(way, colored_ips):
    G = build_graph(way)

    plt.figure(figsize=(15, 15))

    pos = nx.spring_layout(G, scale=0.7)

    # Color nodes based on conditions
    node_colors = []
    for node_ip in G.nodes:
        if node_ip in ["192.168.1.1", "192.168.1.100"]:
            node_colors.append("lightcoral")
        elif node_ip in colored_ips:
            node_colors.append("skyblue")
        else:
            node_colors.append("skyblue")

    nx.draw(G, pos, with_labels=True, node_size=400, node_color=node_colors, font_size=6)
    nx.draw_networkx_edge_labels(G, pos, edge_labels={}, font_size=6)

    plt.title("Dijkstra's Algorithm Network")
    plt.show()

In [ ]:
def initialize_routes():
    way = [Route() for _ in range(routeQtd)]

    for i in range(routeQtd):
        current_router = None
        for j in range(random.randint(5, 10)):  # Random number of routers in each route
            ip = f"192.168.1.{random.randint(2, 99)}"
            last_bin_oc = random.randint(1, 100)
            next_time = random.uniform(0.1, 10.0)
            fitness = 0.0
            router = Router(ip, last_bin_oc, next_time, fitness)

            if current_router is None:
                way[i].firstRouter = router
            else:
                current_router.nextRtd = router

            current_router = router

    return way

In [ ]:
def main():
    way = initialize_routes()

    # Build the graph
    G = build_graph(way)
    source_node = "192.168.1.1"
    target_node = "192.168.1.100"
    # Find the shortest path using Dijkstra's algorithm
    shortest_path = dijkstra_shortest_path(G, source=sourceHost, destination=destinationHost)

    print(f"Shortest Path from {sourceHost} to {destinationHost}: {shortest_path}")

    colored_ips = set(shortest_path)
    plot_routes_with_color(way, colored_ips)

if __name__ == "__main__":
    main()